<a href="https://colab.research.google.com/github/Vinayak-Sharma12/GENAI-II/blob/main/8_Currency_Converted_Using_LLM_and_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q langchain langchain_groq langchain_community requests

### **Tool to fetch live Conversion factor and Convert function** *italicized text*

In [5]:
from langchain_core.tools import tool
import requests
import json

In [9]:
@tool
def conversion_factor(base_currency:str,target_currency:str)->float:
     """This function fetches currency conversion factor between given base currency and target currency"""
     url=f"https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}"
     response=requests.get(url)
     return response.json()

In [11]:
@tool
def convert(base_currency_value:int,conversion_rate:float)->float:
  """given a currency conversion rate this function calculates the target currency value from a given base currency value"""

  return base_currency_value*conversion_rate




## **Tool Binding**

In [12]:
from langchain_groq import ChatGroq
llm=ChatGroq(model='llama-3.1-8b-instant',api_key='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')

In [13]:
llm_with_tools=llm.bind_tools([conversion_factor,convert])

In [26]:
result=llm_with_tools.invoke('What is the USD 10 in INR')

In [27]:
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_s1g3', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'conversion_factor'}, 'type': 'function'}, {'id': 'call_1j8n', 'function': {'arguments': '{"base_currency_value": 10, "conversion_rate": 80.57}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 336, 'total_tokens': 383, 'completion_time': 0.062666667, 'prompt_time': 0.060407298, 'queue_time': 0.522534916, 'total_time': 0.123073965}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4514ade6-a4b3-4867-9d85-85ecb54cde48-0', tool_calls=[{'name': 'conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_s1g3', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10, 'conversion_rate': 80.57}, 'id': 'call_1j8n', 'type': 

In [29]:
result.tool_calls
#Current was 84.57 so this means llm is taking the old one so it means it is running both tools
#And not giving input from first tool to other which should be the case
#And also it is not AI agent we are just integrating tools with lln

[{'name': 'conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_s1g3',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10, 'conversion_rate': 80.57},
  'id': 'call_1j8n',
  'type': 'tool_call'}]

### **Injected Tool Argument**

In [30]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [32]:
@tool
def convert(base_currency_value:int,conversion_rate:Annotated[float,InjectedToolArg])->float:
  """given a currency conversion rate this function calculates the target currency value from a given base currency value"""

  return base_currency_value*conversion_rate




In [60]:
llm_with_tools=llm.bind_tools([conversion_factor,convert])

In [61]:
result=llm_with_tools.invoke('What is the USD 10 in INR')

In [62]:
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mzaa', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'conversion_factor'}, 'type': 'function'}, {'id': 'call_y4ef', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 326, 'total_tokens': 364, 'completion_time': 0.050666667, 'prompt_time': 0.031708085, 'queue_time': 0.411176132, 'total_time': 0.082374752}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-95679b43-9678-4d26-898f-0e530ce0a577-0', tool_calls=[{'name': 'conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_mzaa', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10}, 'id': 'call_y4ef', 'type': 'tool_call'}], usage_metadata={'input_tokens': 326, 

In [63]:
result.tool_calls

[{'name': 'conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_mzaa',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_y4ef',
  'type': 'tool_call'}]

In [64]:
result.tool_calls[0]['args']['target_currency']

'INR'

In [65]:
result.tool_calls[0]['args']['base_currency']

'USD'

In [66]:
output=conversion_factor.invoke({'base_currency':result.tool_calls[0]['args']['base_currency'],'target_currency':result.tool_calls[0]['args']['target_currency']})

In [67]:
output

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1745971201,
 'time_last_update_utc': 'Wed, 30 Apr 2025 00:00:01 +0000',
 'time_next_update_unix': 1746057601,
 'time_next_update_utc': 'Thu, 01 May 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.2184}

In [68]:
result.tool_calls[1]['args']['conversion_rate']=output['conversion_rate']

In [69]:
result.tool_calls

[{'name': 'conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_mzaa',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10, 'conversion_rate': 85.2184},
  'id': 'call_y4ef',
  'type': 'tool_call'}]

In [71]:
final_result=convert.invoke('base_conversion')

TypeError: unhashable type: 'dict'

### *AI Agent*

In [74]:
from langchain.agents import initialize_agent,AgentType
agent_executor=initialize_agent(
    tools=[conversion_factor,convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [75]:
user_query = "Convert 1000 USD to INR"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Action: 
```
{
  "action": "conversion_factor",
  "action_input": {
    "base_currency": "USD",
    "target_currency": "INR"
  }
}
```

Observation: {'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1745971201, 'time_last_update_utc': 'Wed, 30 Apr 2025 00:00:01 +0000', 'time_next_update_unix': 1746057601, 'time_next_update_utc': 'Thu, 01 May 2025 00:00:01 +0000', 'base_code': 'USD', 'target_code': 'INR', 'conversion_rate': 85.2184}
Thought:Action: 
```
{
  "action": "convert",
  "action_input": {
    "base_currency_value": 1000,
    "conversion_rate": 85.2184
  }
}
```
Observation: 85218.40000000001
Thought:Action: 
```
{
  "action": "Final Answer",
  "action_input": "The conversion of 1000 USD to INR is 85218.40000000001"
}
```

> Finished chain.


In [76]:
response

{'input': 'Convert 1000 USD to INR',
 'output': 'The conversion of 1000 USD to INR is 85218.40000000001'}